In [ ]:
from datetime import date
import calendar as c
import pandas as pd
import numpy as np

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 1500)
pd.options.display.max_colwidth = 100000

#### Para testear las funciones, levanto los dataframes originales

In [ ]:
df_pageviews = pd.read_csv('data/pageviews.csv', parse_dates=['FEC_EVENT'])
df_devicedata = pd.read_csv('data/device_data.csv', parse_dates=["FEC_EVENT"])
year = 2018
mes_snapshot = 9

def universo_train(df):
    universo = pd.DataFrame()
    universo['user_id'] = df.USER_ID.unique()
    universo['snapshot_mes'] = mes_snapshot
    return universo

universo = universo_train(df_pageviews)

In [ ]:
def _sum_campo_user(df_pageviews, campo):
    temp = pd.crosstab(df_pageviews.USER_ID, df_pageviews[campo]) 
    temp.columns = [campo + "_" + str(v) for v in temp.columns]
    temp = _df_refactor(temp)
    return temp

In [ ]:
def _df_refactor(df_temp):
    df_union = pd.merge(universo, df_temp, how='left', left_on=['user_id'], right_on=['USER_ID'])
    df_union = df_union.drop(['snapshot_mes'], axis=1)
    df_union = df_union.sort_values('user_id')
    return df_union

In [ ]:
def _sum_campo_user_between(df_pageviews, fecha_desde, fecha_hasta, campo):
    df = df_pageviews[fecha_desde <= df_pageviews['FEC_EVENT']]
    df = df[df['FEC_EVENT'] <= fecha_hasta]
    return _sum_campo_user(df, campo)

In [ ]:
def _sum_total_user(df_pageviews, campo):
    temp = df_pageviews.groupby("USER_ID")[campo].sum().to_frame().reset_index()
    temp = _df_refactor(temp)
    temp = temp.drop(['USER_ID'], axis=1)
    return temp

In [ ]:
def _sum_total_user_between(df_pageviews, fecha_desde, fecha_hasta, campo):
    df = df_pageviews[fecha_desde <= df_pageviews['FEC_EVENT']]
    df = df[df['FEC_EVENT'] <= fecha_hasta]
    return _sum_total_user(df, campo)

In [ ]:
def _df_complete_columns(df_complete_original, df_incomplete):
    df_complete = df_complete_original.copy()
    df_complete[:] = 0
    df = pd.DataFrame()
    for a_column in df_complete:
        if a_column in df_incomplete.columns:
            df[a_column] = df_incomplete[a_column]
        else:
            df[a_column] = df_complete[a_column]
    return df

In [ ]:
def _df_repeat_columns(df_complete_original, df_incomplete):
    users = df_complete_original['user_id']
    df_complete = df_complete_original.copy()
    df = df_complete[[df_complete.columns[i] for i in range(df_complete.shape[1]-1)]]
    df['user_id'] = users
    return df

In [ ]:
def _sum_user_campo_all(df_pageviews, mes_snapshot, campo):
    fecha_desde_3m = pd.Timestamp(year, mes_snapshot - 2, 1)
    fecha_desde_1m = pd.Timestamp(year, mes_snapshot, 1)
    fecha_desde_15d = pd.Timestamp(year, mes_snapshot, c.monthrange(year, mes_snapshot)[1] - 15)
    fecha_desde_7d = pd.Timestamp(year, mes_snapshot, c.monthrange(year, mes_snapshot)[1] - 7)
    fecha_desde_1d = pd.Timestamp(year, mes_snapshot, c.monthrange(year, mes_snapshot)[1] - 1)
    fecha_hasta_last_day = pd.Timestamp(year, mes_snapshot, c.monthrange(year, mes_snapshot)[1])
    
    df_3m = _sum_campo_user_between(df_pageviews, fecha_desde_3m, fecha_hasta_last_day, campo)
    df_1m = _sum_campo_user_between(df_pageviews, fecha_desde_1m, fecha_hasta_last_day, campo)
    df_15d = _sum_campo_user_between(df_pageviews, fecha_desde_15d, fecha_hasta_last_day, campo)
    df_7d = _sum_campo_user_between(df_pageviews, fecha_desde_7d, fecha_hasta_last_day, campo)
    df_1d =_sum_campo_user_between(df_pageviews, fecha_desde_1d, fecha_hasta_last_day, campo)
    df_ever =_sum_campo_user(df_pageviews, campo)

    tot_3m = _sum_total_user_between(df_pageviews, fecha_desde_3m, fecha_hasta_last_day, campo)
    tot_1m = _sum_total_user_between(df_pageviews, fecha_desde_1m, fecha_hasta_last_day, campo)
    tot_15d = _sum_total_user_between(df_pageviews, fecha_desde_15d, fecha_hasta_last_day, campo)
    tot_7d = _sum_total_user_between(df_pageviews, fecha_desde_7d, fecha_hasta_last_day, campo)    
    tot_1d = _sum_total_user_between(df_pageviews, fecha_desde_1d, fecha_hasta_last_day, campo)
    tot_ever = _sum_total_user(df_pageviews, campo)
    
    df_3m =_df_complete_columns(df_ever, df_3m)
    df_1m =_df_complete_columns(df_ever, df_1m)
    df_15d =_df_complete_columns(df_ever, df_15d)
    df_7d =_df_complete_columns(df_ever, df_7d)
    df_1d =_df_complete_columns(df_ever, df_1d)
    
    return df_3m, df_1m, df_15d, df_7d, df_1d, df_ever, tot_3m, tot_1m, tot_15d, tot_7d, tot_1d, tot_ever

In [ ]:
def _ratio_campo_df(df_numerador, df_denominador, name):
    #Eliminar columnar user_id para que no la divida
    users = df_numerador['user_id']
    df_numerador = df_numerador.drop(['user_id'], axis = 1)
    df_denominador = df_denominador.drop(['user_id'], axis = 1)
    
    #División element-wise
    temp = df_numerador.div(df_denominador, axis=0, fill_value = 0)
    temp.columns = [name + "_" + str(v) for v in temp.columns]

    #Agrega columna user_id
    temp['user_id'] = users

    #Completa Nan
    temp = temp.fillna(-999)
    return temp

In [ ]:
def _ratio_campo_all(universo, df_pageviews, mes_snapshot, campo):
    df_3m, df_1m, df_15d, df_7d, df_1d, df_ever, tot_3m, tot_1m, tot_15d, tot_7d, tot_1d, tot_ever = _sum_user_campo_all(df_pageviews, mes_snapshot, campo)

    tot_3m_r = _df_repeat_columns(df_ever, tot_3m)
    tot_1m_r = _df_repeat_columns(df_ever, tot_1m)
    tot_15d_r = _df_repeat_columns(df_ever, tot_15d)
    tot_7d_r = _df_repeat_columns(df_ever, tot_7d)
    tot_1d_r = _df_repeat_columns(df_ever, tot_1d)
    tot_ever_r = _df_repeat_columns(df_ever, tot_ever)
    
    print('llegue 1/9')
    ratio_preferencia_3m = _ratio_campo_df(df_3m, tot_3m_r, 'ratio_preferencia_3m')
    ratio_preferencia_1m = _ratio_campo_df(df_1m, tot_1m_r, 'ratio_preferencia_1m')
    ratio_preferencia_15d = _ratio_campo_df(df_15d, tot_15d_r, 'ratio_preferencia_15d')
    ratio_preferencia_7d = _ratio_campo_df(df_7d, tot_7d_r, 'ratio_preferencia_7d')
    ratio_preferencia_1d = _ratio_campo_df(df_1d, tot_1d_r, 'ratio_preferencia_1d')
    print('llegue 2/9')
    ratio_interes_3m = _ratio_campo_df(df_3m, df_ever, 'ratio_interes_3m')
    ratio_interes_1m = _ratio_campo_df(df_1m, df_ever, 'ratio_interes_1m')
    ratio_interes_15d = _ratio_campo_df(df_15d, df_ever, 'ratio_interes_15d')
    ratio_interes_7d = _ratio_campo_df(df_7d, df_ever, 'ratio_interes_7d')
    ratio_interes_1d = _ratio_campo_df(df_1d, df_ever, 'ratio_interes_1d')
    print('llegue 3/9')
    ratio_interes_reciente_1m = _ratio_campo_df(df_1m, df_3m, 'ratio_interes_reciente_1m')
    ratio_interes_reciente_15d = _ratio_campo_df(df_15d, df_3m, 'ratio_interes_reciente_15d')
    ratio_interes_reciente_7d = _ratio_campo_df(df_7d, df_3m, 'ratio_interes_reciente_7d')
    ratio_interes_reciente_1d = _ratio_campo_df(df_1d, df_3m, 'ratio_interes_reciente_1d')
    print('llegue 4/9')
    ratio_actividad_3m = _ratio_campo_df(tot_3m, tot_ever, 'ratio_actividad_3m')
    ratio_actividad_1m = _ratio_campo_df(tot_1m, tot_ever, 'ratio_actividad_1m')
    ratio_actividad_15d = _ratio_campo_df(tot_15d, tot_3m, 'ratio_actividad_15d')
    ratio_actividad_7d = _ratio_campo_df(tot_7d, tot_3m, 'ratio_actividad_7d')
    ratio_actividad_1d = _ratio_campo_df(tot_1d, tot_3m, 'ratio_actividad_1d')
    print('llegue 5/9')
    df_union = pd.merge(universo, ratio_preferencia_3m, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_preferencia_1m, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_preferencia_15d, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_preferencia_7d, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_preferencia_1d, how='left', left_on=['user_id'], right_on=['user_id'])
    print('llegue 6/9')
    df_union = pd.merge(df_union, ratio_interes_3m, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_interes_1m, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_interes_15d, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_interes_7d, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_interes_1d, how='left', left_on=['user_id'], right_on=['user_id'])
    print('llegue 7/9')
    df_union = pd.merge(df_union, ratio_interes_reciente_1m, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_interes_reciente_15d, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_interes_reciente_7d, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_interes_reciente_1d, how='left', left_on=['user_id'], right_on=['user_id'])
    print('llegue 8/9')
    df_union = pd.merge(df_union, ratio_actividad_3m, how='left', left_on=['user_id'], right_on=['user_id'] )
    df_union = pd.merge(df_union, ratio_actividad_1m, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_actividad_15d, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_actividad_7d, how='left', left_on=['user_id'], right_on=['user_id'])
    df_union = pd.merge(df_union, ratio_actividad_1d, how='left', left_on=['user_id'], right_on=['user_id'])
    print('llegue 9/9')
    return df_union

In [ ]:
def ratio_page_all(universo, df_pageviews, mes_snapshot):
    campo = 'PAGE'
    return _ratio_campo_all(universo, df_pageviews, mes_snapshot, campo)

In [ ]:
def ratio_ccategory_all(universo, df_pageviews, mes_snapshot):
    campo = 'CONTENT_CATEGORY'
    return _ratio_campo_all(universo, df_pageviews, mes_snapshot, campo)

In [ ]:
def ratio_cctop_all(universo, df_pageviews, mes_snapshot):
    campo = 'CONTENT_CATEGORY_TOP'
    return _ratio_campo_all(universo, df_pageviews, mes_snapshot, campo)

In [ ]:
def ratio_ccbottom_all(universo, df_pageviews, mes_snapshot):
    campo = 'CONTENT_CATEGORY_BOTTOM'
    return _ratio_campo_all(universo, df_pageviews, mes_snapshot, campo)

In [ ]:
def ratio_site_id_all(universo, df_pageviews, mes_snapshot):
    campo = 'SITE_ID'
    return _ratio_campo_all(universo, df_pageviews, mes_snapshot, campo)

In [ ]:
def ratio_mobile_device_all(universo, df_devicedata, mes_snapshot):
    campo = 'IS_MOBILE_DEVICE'
    return _ratio_campo_all(universo, df_devicedata, mes_snapshot, campo)

In [ ]:
def ratio_connection_speed_all(universo, df_devicedata, mes_snapshot):
    campo = 'CONNECTION_SPEED'
    return _ratio_campo_all(universo, df_devicedata, mes_snapshot, campo)

In [ ]:
def ratio_on_site_search_term_all(universo, df_devicedata, mes_snapshot):
    campo = 'ON_SITE_SEARCH_TERM'
    return _ratio_campo_all(universo, df_devicedata, mes_snapshot, campo)